# Week 6 Assignment: Extending the Voice Agent with Function Calling.
This week, we dive into a crucial capability of modern AI agents: function calling. Real-world agents are not limited to generating text—they interact with tools, run computations, query databases, and more. In this assignment, you will extend the multi-turn voice agent you built in Week 3 with the ability to automatically execute tools based on natural language commands.

Using Llama 3 as your core LLM, you’ll teach the model to recognize when a user wants to search arXiv papers or perform a math calculation, and respond by outputting a structured function call. Your agent will then parse that function call, execute the appropriate tool, and return the result to the user via text-to-speech. This is a major step toward building a fully autonomous research assistant that can act on intent—not just reply with facts.



## 📚 Learning Objectives

* **Function Calling with LLMs:** Learn how to use function/tool calling by prompting Llama 3 to output structured calls (e.g., JSON) for external functions.
* **Intent Parsing and Tool Mapping:** Practice parsing user queries to determine the intent (e.g., search a database or perform a calculation) and mapping that intent to specific tool functions like `search_arxiv(query)` and `calculate(expression)`.
* **Integrating Tools into the Agent:** Extend the Week 3 multi-turn voice agent pipeline (ASR → LLM → TTS) so that the LLM can trigger code execution. The agent should automatically call the right function based on the LLM’s output, and then speak the returned result.



## Project Design

Reuse the voice-chat pipeline from Week 3 and enhance the LLM step to support calling external tools. The key tasks include:

* **Tool Functions:** Implement two helper functions:

  * `search_arxiv(query: str) -> str`: Simulates or performs an arXiv search and returns a relevant passage or summary for the query.
  * `calculate(expression: str) -> str`: Evaluates a mathematical expression (using `sympy` or `eval`) and returns the result as text.

* **Prompt Engineering:** Modify the Llama 3 system/user prompts so the model knows to generate structured function-call outputs when appropriate. For example, instruct it that if the user’s question can be answered by searching arXiv or doing math, it should output a JSON-like call, such as:

  ```json
  {"function": "calculate", "arguments": {"expression": "2+2"}}
  ```

  or

  ```json
  {"function": "search_arxiv", "arguments": {"query": "quantum entanglement"}}
  ```

  Otherwise it should respond normally in text.

* **Detecting and Calling Tools:** After the LLM generates a response, check if it is a function call. Parse the JSON output from the LLM to extract the function name and arguments. If it is a call, invoke the corresponding Python function (`search_arxiv` or `calculate`) with those arguments and capture its result. Use this result as the assistant’s reply (to be spoken by TTS). If the LLM output is normal text, just use it as the assistant’s response without calling any function.

* **Fallback Behavior:** Ensure the voice agent handles all cases. If the LLM’s output cannot be parsed as a function call (or if the named function is unknown), fall back to replying with a standard text response or an error message as appropriate.

## Starter Code

We provide snippets to help you get started:


In [ ]:
# Function tool stubs (starter implementations)
def search_arxiv(query: str) -> str:
    """
    Simulate an arXiv search or return a dummy passage for the given query.
    In a real system, this might query the arXiv API and extract a summary.
    """
    # Example placeholder implementation:
    return f"[arXiv snippet related to '{query}']"

def calculate(expression: str) -> str:
    """
    Evaluate a mathematical expression and return the result as a string.
    """
    try:
        from sympy import sympify
        result = sympify(expression)  # use sympy for safe evaluation
        return str(result)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
# Dialogue engine: function-routing logic
import json

def route_llm_output(llm_output: str) -> str:
    """
    Route LLM response to the correct tool if it's a function call, else return the text.
    Expects LLM output in JSON format like {'function': ..., 'arguments': {...}}.
    """
    try:
        output = json.loads(llm_output)
        func_name = output.get("function")
        args = output.get("arguments", {})
    except (json.JSONDecodeError, TypeError):
        # Not a JSON function call; return the text directly
        return llm_output

    if func_name == "search_arxiv":
        query = args.get("query", "")
        return search_arxiv(query)
    elif func_name == "calculate":
        expr = args.get("expression", "")
        return calculate(expr)
    else:
        return f"Error: Unknown function '{func_name}'"


In [ ]:
# Example FastAPI endpoint (sketch)
from fastapi import FastAPI
app = FastAPI()

@app.post("/api/voice-query/")
async def voice_query_endpoint(request: dict):
    # Assume request has 'text': the user's query string
    user_text = request.get("text", "")
    # Call Llama 3 model (instructed to output function calls when needed)
    llm_response = llama3_chat_model(user_text)
    # Process LLM output and possibly call tools
    reply_text = route_llm_output(llm_response)
    # Convert reply_text to speech (TTS) and return it
    return {"response": reply_text}


The above code outlines where to plug in your LLM call and audio I/O. Integrate the function-calling logic into your existing voice agent framework.



## Deliverables

* **Codebase:** Submit your updated voice agent code with function calling integration. Document any new modules or changes clearly.
* **Test Logs:** Provide sample logs for at least three queries, showing:

  1. The user’s query text.
  2. The raw LLM response (JSON function call or normal text).
  3. Any function call made and its output.
  4. The final assistant response.
* **Demo Video:** A 1–2 minute demo of the voice agent. Show the agent handling:

  * A math query (invoking `calculate`).
  * An arXiv search query (invoking `search_arxiv`).
  * A normal query (no function call).

## Exploration Tips

* **Extend Tools:** Try adding new tools (e.g. a weather lookup or translation). Define their function signatures and integrate them into your agent.
* **Tool Registry:** Create a dictionary or registry of function names to callables to simplify routing logic when you have multiple tools.
* **Other LLMs:** Experiment with other models that support function calling (e.g. GPT-4 with its function calling API). Compare how their output format and reliability differ from Llama 3.
* **Error Handling:** Make sure your agent handles invalid inputs gracefully (e.g. a malformed math expression should not crash the agent).
* **Chained Calls (Advanced):** As a challenge, allow the agent to use one tool’s output as context for another. For example, it could `search_arxiv` for a value and then `calculate` something with that value.
